<span STYLE="font-size:150%"> 
    Make calibration curve from standard samples
</span>

Docker image: gnasello/datascience-env:2023-03-30 \
Latest update: 31 March 2023

# Load libraries

In [ ]:
library(ggplot2)
library(latex2exp)
library(ggpubr)
library(yaml)
library(readxl)

Clone the [r_utils library](https://github.com/gabnasello/r_utils.git) from GitHub in the same folder of this script. 

You can simply running the following command in a new terminal (open it from JupyterLab):

`git clone https://github.com/gabnasello/r_utils.git`

How to Reuse Functions That You Create In Scripts, [tutorial](https://www.earthdatascience.org/courses/earth-analytics/multispectral-remote-sensing-data/source-function-in-R/)

In [ ]:
source("r_utils/ggplot_utils.R")

# Reading data

In [ ]:
filename <- "data/2023-03-28_GN011.xlsx"

In [ ]:
df <- read_excel(filename)

# Import Excel Data into R Dataframe
df <- data.frame(df)
head(df)

Remove df rows where the `Well.ID` columns is `NA`

In [ ]:
df <- df[complete.cases(df$Well.ID),]
head(df)

# Select standard samples

In [ ]:
prefix <- 'STD'

In [ ]:
condition <- startsWith(df$Well.ID, prefix)

df_std <- df[condition,]
df_std

# Subtract blank from `Mean` values

In [ ]:
df_std$Mean_blnk <- df_std$Mean - df_std[df_std$Conc.Dil == 0, 'Mean']
df_std

Subtract blank from `Mean` values of the entire dataset

In [ ]:
df$Mean_blnk <- df$Mean - df_std[df_std$Conc.Dil == 0, 'Mean']
head(df)

Overwrite initial dataset after subtracting the blank value

In [ ]:
new_filename <- paste(tools::file_path_sans_ext(filename), '.csv', sep='')
new_filename

In [ ]:
write.csv(df, new_filename, row.names=FALSE)

# Plot calibration data

In [ ]:
p <- ggplot(df_std, aes(x=Conc.Dil, y=Mean_blnk)) + geom_point(size=1.5)

img <- ggplotMinAethetics(p, width=4, height=3.25,
                          title='Calibration curve',
                          xlabel=TeX('Concentration [$\\mu$g/ml]'), 
                          ylabel='Emission (AU)', 
                          scale_color='npg')
img

In [ ]:
p <- ggplot(df_std, aes(x=Mean_blnk, y=Conc.Dil)) + geom_point(size=1.5)

img <- ggplotMinAethetics(p, width=4, height=3.25,
                          title='Calibration curve',
                          ylabel=TeX('Concentration [$\\mu$g/ml]'), 
                          xlabel='Emission (AU)', 
                          scale_color='npg')
img

# Linear regression

Set the **intercept** of the linear model to **zero**

In [ ]:
# Build the linear model
model  <- lm(Conc.Dil ~ Mean_blnk + 0, data = df_std)
summary(model)

In [ ]:
p <- ggplot(df_std, aes(x=Mean_blnk, y=Conc.Dil)) + geom_point(size=1.5) + 
     geom_abline(slope = coef(model)[["Mean_blnk"]], 
                 intercept = 0, color='turquoise4', linewidth=1.1)

img <- ggplotMinAethetics(p, width=4, height=3.25,
                          title='Calibration curve',
                          ylabel=TeX('Concentration ($\\mu$g/ml)'), 
                          xlabel='Emission (AU)', 
                          scale_color='npg') +
       ylim(0, 58)
img

# Save model

In [ ]:
saveRDS(model, "calibration.rds")